In [3]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

cols = list(pd.read_csv("cancer.csv", nrows =1))
data = pd.read_csv('cancer.csv', usecols=[i for i in cols if i != 'id' and i != 'Unnamed: 32'])
data.head(-5)

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.26540,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.18600,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.24300,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.25750,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.16250,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559,B,11.51,23.93,74.52,403.5,0.09261,0.10210,0.11120,0.04105,0.1388,...,12.48,37.16,82.28,474.2,0.12980,0.25170,0.3630,0.09653,0.2112,0.08732
560,B,14.05,27.15,91.38,600.4,0.09929,0.11260,0.04462,0.04304,0.1537,...,15.30,33.17,100.20,706.7,0.12410,0.22640,0.1326,0.10480,0.2250,0.08321
561,B,11.20,29.37,70.67,386.0,0.07449,0.03558,0.00000,0.00000,0.1060,...,11.92,38.30,75.19,439.6,0.09267,0.05494,0.0000,0.00000,0.1566,0.05905
562,M,15.22,30.62,103.40,716.9,0.10480,0.20870,0.25500,0.09429,0.2128,...,17.52,42.79,128.70,915.0,0.14170,0.79170,1.1700,0.23560,0.4089,0.14090


In [4]:
data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

data_dev = data[0:400].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[400:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

In [126]:
class NeuralNetwork:
    
    def __init__(self):
        np.random.seed(10)
        self.W1 = np.random.rand(2, 30) - 0.5
        self.b1 = np.random.rand(2, 1) - 0.5
        
        self.W2 = np.random.rand(2, 2) - 0.5
        self.b2 = np.random.rand(2, 1) - 0.5
        
    def __ReLU(self, Z):
        return np.maximum(Z, 0)

    def __softmax(self, Z):
        Z = Z.astype(float)
        A = np.exp(Z - np.max(Z)) / sum(np.exp(Z - np.max(Z)))
        return A
    
    def __forward(self, x):
        Z1 = self.W1.dot(x) + self.b1
        A1 = self.__ReLU(Z1)
        Z2 = self.W2.dot(A1) + self.b2
        A2 = self.__softmax(Z2)
        return Z1, A1, Z2, A2
    
    def __ReLU_deriv(self, Z):
        return Z > 0

    def __label_encoder(self, Y):
        encoder = dict(zip(np.unique(Y), range(len(np.unique(Y)))))
        labels = []
        for i in Y:
            labels.append(encoder[i])
        
        return np.array(labels)
    
    def __one_hot(self, Y):
        Y = self.__label_encoder(Y)
        one_hot_Y = np.zeros((Y.size, Y.max() + 1))
        one_hot_Y[np.arange(Y.size), Y] = 1
        one_hot_Y = one_hot_Y.T
        return one_hot_Y

    def __backward(self, Z1, A1, Z2, A2, x, y, learning_rate):
        one_hot_Y = self.__one_hot(y)

        dZ2 = A2 - one_hot_Y
        dW2 = 1 / m * dZ2.dot(A1.T)
        db2 = 1 / m * np.sum(dZ2)

        dZ1 = self.W2.T.dot(dZ2) * self.__ReLU_deriv(Z1)
        dW1 = 1 / m * dZ1.dot(x.T)
        db1 = 1 / m * np.sum(dZ1)
        
        self.W1 = self.W1 - learning_rate * dW1
        self.b1 = self.b2 - learning_rate * db1
        self.W2 = self.W2 - learning_rate * dW2  
        self.b2 = self.b2 - learning_rate * db2
                
    def __get_predictions(self, A2):
        return np.argmax(A2, 0)

    def __get_accuracy(self, predictions, Y):
        #print(Y)
        return np.sum(predictions == Y) / Y.size
    
    def __predict(self, X):
        _, _, _, A2 = self.__forward(X)
        predictions = self.__get_predictions(A2)
        return predictions

    def fit(self, X, Y, learning_rate, epochs):
        for i in range(epochs):
            Z1, A1, Z2, A2 = self.__forward(X)
            self.__backward(Z1, A1, Z2, A2, X, Y, learning_rate)
            if i % 10 == 0:
                print('Predicted', self.__predict(X))
                print('Y:', self.__one_hot(Y))
                print(self.__get_accuracy(self.__predict(X), self.__one_hot(Y)[0]))
                #print(self.__forward(X))
                print('----------------')

In [128]:
if __name__ == '__main__' and '__file__' not in globals():
    nn = NeuralNetwork()
    nn.fit(X_train, Y_train, 0.001, 500)

Predicted [0 1 0 1 0 0 0 0 1 0 0 1 1 0 0 1 0 0 1 0 1 1 1 1 0 0 1 0 0 1 0 0 1 1 0 0 1
 1 0 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1
 1 0 1 0 0 1 0 1 1 1 0 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 0 1 0 1 1 0 1 1 1
 0 1 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 0 1 0 1 1
 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 0 1 1 0 0 0]
Y: [[0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1.
  0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1.
  1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0.
  1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1.
  1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
  1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0.
  1.]
 [1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1

Predicted [0 1 1 1 0 1 0 0 1 1 1 1 1 0 0 1 0 0 1 0 1 1 1 1 0 1 1 0 1 1 0 1 1 1 0 1 1
 1 0 0 0 0 1 1 0 1 0 1 1 0 1 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 0 0 1 0 1 1 1 1
 1 1 1 0 0 1 0 1 1 1 1 1 1 0 0 0 1 0 1 1 0 1 0 0 1 1 1 0 1 1 0 1 1 0 1 1 1
 0 1 1 1 1 1 1 0 1 1 0 1 0 0 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 1 1 1 1
 1 1 1 0 1 0 0 0 0 1 1 1 1 1 0 1 1 1 0 0 1]
Y: [[0. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1.
  0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 1.
  1. 0. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0.
  1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1.
  1. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 0. 0.
  0. 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 0.
  1. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 0. 1. 1. 1. 0. 0.
  1.]
 [1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0. 0. 0. 0.
  1. 0. 0. 1. 0. 0. 1. 1. 0. 0. 1. 0. 0. 0. 1